In [16]:
# install required package
import subprocess
import sys

print("installing fuzzywuzzy...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "fuzzywuzzy", "-q"])
print("installation complete\n")

# Mounting Drive for quick sharing and checking of final file
from google.colab import drive
drive.mount('/content/drive')

# importing all required libraries - sep to environemtn
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("\nFIFA Master Dataset Creation")
print("Players Active 2019-2023")
print("="*60)

# Data coonfiguration and initial set up
print("\nStage 1: Configuration and setup")

# defining the data sources - 5 years agreed
fifa_files = {
    '2019': 'https://raw.githubusercontent.com/fern-1210/IronHack-w3-Modual1-Project3/refs/heads/main/Data/Raw/FIFA%202017-2023%20data%20raw/FIFA19_official_data.csv',
    '2020': 'https://raw.githubusercontent.com/fern-1210/IronHack-w3-Modual1-Project3/refs/heads/main/Data/Raw/FIFA%202017-2023%20data%20raw/FIFA20_official_data.csv',
    '2021': 'https://raw.githubusercontent.com/fern-1210/IronHack-w3-Modual1-Project3/refs/heads/main/Data/Raw/FIFA%202017-2023%20data%20raw/FIFA21_official_data.csv',
    '2022': 'https://raw.githubusercontent.com/fern-1210/IronHack-w3-Modual1-Project3/refs/heads/main/Data/Raw/FIFA%202017-2023%20data%20raw/FIFA22_official_data.csv',
    '2023': 'https://raw.githubusercontent.com/fern-1210/IronHack-w3-Modual1-Project3/refs/heads/main/Data/Raw/FIFA%202017-2023%20data%20raw/FIFA23_official_data.csv',
}

# columns to remove from final dataset as not needed for task
columns_to_delete = ['Photo', 'Flag', 'Potential', 'Club Logo', 'International Reputation',
                     'Weak Foot', 'Skill Moves', 'Real Face', 'Jersey Number', 'Kit Number',
                     'Joined', 'Loaned From', 'Contract Valid Until', 'Best Position',
                     'Best Overall Rating', 'Special', 'Release Clause', 'Value']

print(f"Data sources configured: {len(fifa_files)} years (2019-2023)")
print(f"Columns to be removed: {len(columns_to_delete)}")

# player position mapping - converting the existing codes to readable names for further analysis
position_mapping = {
    'GK': 'Goalkeeper',
    'CB': 'Centre Back',
    'LB': 'Left Back',
    'RB': 'Right Back',
    'LWB': 'Left Wing Back',
    'RWB': 'Right Wing Back',
    'RCB': 'Right Centre Back',
    'LCB': 'Left Centre Back',
    'CDM': 'Centre Defensive Midfield',
    'CM': 'Centre Midfield',
    'CAM': 'Centre Attacking Midfield',
    'LM': 'Left Midfield',
    'RM': 'Right Midfield',
    'RDM': 'Right Defensive Midfield',
    'LDM': 'Left Defensive Midfield',
    'RCM': 'Right Centre Midfield',
    'LCM': 'Left Centre Midfield',
    'LW': 'Left Wing',
    'RW': 'Right Wing',
    'LF': 'Left Forward',
    'RF': 'Right Forward',
    'CF': 'Centre Forward',
    'ST': 'Striker',
    'LS': 'Left Striker',
    'RS': 'Right Striker',
    'SUB': 'Substitute',
    'RES': 'Reserve'
}

print(f"player position mapping configured: {len(position_mapping)} positions")
print("Initial work complete - all data ready\n")

# STAGE two: Data Loading and initial cleaning process
print("STAGE 2 - Sourcing all raw data from our downloaded files on repo on git")

def clean_text(text):
    """Remove HTML tags and normalise whitespace"""
    if pd.isna(text):
        return text
    text = str(text)

    # Simple HTML removal - keep removing anything between < and >
    while '<' in text and '>' in text:
        start = text.find('<')
        end = text.find('>', start)
        if end > start:
            text = text[:start] + text[end+1:]
        else:
            break

    # Clean up whitespace
    text = ' '.join(text.split())
    return text.strip()

def extract_position_code(position_text):
    """Extract position code from HTML and map to readable name"""
    if pd.isna(position_text):
        return 'Unknown'

    # extract the position code (e.g., 'LB' from '<span class="pos pos7">LB')
    position_text = str(position_text)

    # if there's any other HTMl code lets extract just the position code
    if '>' in position_text:
        position_code = position_text.split('>')[-1].strip()
    else:
        position_code = position_text.strip()

    # map to more readable name and remove brackets and player number from field
    return position_mapping.get(position_code, position_code)

def format_player_name(name):
    """Format player name as 'F. Surname', removing any leading numbers and brackets"""
    if pd.isna(name):
        return name

    name = str(name).strip()

    # Remove anything in brackets - just take everything before the first bracket
    if '(' in name:
        name = name.split('(')[0]
    if '[' in name:
        name = name.split('[')[0]

    # Remove leading numbers by skipping over digits and spaces at the start
    while len(name) > 0 and (name[0].isdigit() or name[0] == ' '):
        name = name[1:]

    name = name.strip()

    # Check if already in format "F. Surname"
    if len(name) > 2 and name[1] == '.' and ' ' in name:
        return name

    # Split into parts
    parts = name.split()

    if len(parts) == 0:
        return name
    elif len(parts) == 1:
        return f"{parts[0][0]}. {parts[0][1:]}" if len(parts[0]) > 1 else parts[0]
    else:
        first_initial = parts[0][0]
        surname = parts[-1]
        return f"{first_initial}. {surname}"

all_years_data = {}
total_players_loaded = 0

for year, filepath in fifa_files.items():
    print(f"\nLoading FIFA {year}...")
    df = pd.read_csv(filepath, encoding='utf-8', low_memory=False)

    initial_rows = len(df)
    initial_cols = len(df.columns)

    print(f"  raw data: {initial_rows:,} rows, {initial_cols} columns")

    # clean all text columns
    text_columns = df.select_dtypes(include=['object']).columns
    cleaned_count = 0
    for col in text_columns:
        df[col] = df[col].apply(clean_text)
        cleaned_count += 1

    print(f"  cleaned {cleaned_count} text columns (removed HTML tags)")

    # formatting player names
    if 'Name' in df.columns:
        df['Name'] = df['Name'].apply(format_player_name)
        print(f"  formatted player names to 'F. Surname' format")

    # converting position to readable format
    if 'Position' in df.columns:
        df['Position'] = df['Position'].apply(extract_position_code)
        print(f"  converted position codes to readable names")

    # adding year identifier
    df['Year'] = year

    all_years_data[year] = df
    total_players_loaded += len(df)
    print(f"  FIFA {year} loaded successfully")

print(f"\nStage 2 complete - {total_players_loaded:,} total player records loaded across all years\n")

# STAGE 3 Initial Data Assessment
print("STAGE Three data quality assessment")

print("\nAnalysing 2023 dataset as reference")
sample_df = all_years_data['2023'].copy()

print(f"Total columns before cleaning: {len(sample_df.columns)}")

# remove all of the columns we want to delete
columns_removed = 0
for col in columns_to_delete:
    if col in sample_df.columns:
        sample_df = sample_df.drop(col, axis=1)
        columns_removed += 1

print(f"Columns that should be removed from final dataset: {columns_removed}")
print(f"Remaining columns for analysis: {len(sample_df.columns)}")

# Calculate any missing data
missing_before = sample_df.isna().sum()
missing_pct_before = (missing_before / len(sample_df) * 100).round(2)

missing_report_before = pd.DataFrame({
    'column': missing_before.index,
    'missing_count': missing_before.values,
    'missing_percentage': missing_pct_before.values
}).sort_values('missing_percentage', ascending=False)

print("\nTop 15 columns with missing data (before cleaning):")
print(missing_report_before.head(15).to_string(index=False))

total_cells_before = len(sample_df) * len(sample_df.columns)
total_missing_before = sample_df.isna().sum().sum()
overall_pct_before = (total_missing_before / total_cells_before) * 100

print(f"\nOverall data completeness:")
print(f"  Total cells: {total_cells_before:,}")
print(f"  Missing cells: {total_missing_before:,}")
print(f"  Missing percentage: {overall_pct_before:.2f}%")
print(f"  Complete data: {100 - overall_pct_before:.2f}%")

print("\nstage 3 done - initial assessment finished\n")

# Stage 4 Player matching Across all Years 2019-2023
print("Stage 4 - Matching same players across all years (2019-2023)")

print("\nThis stage identifies which players appear in all of the five years of data being assessed")
print("Method: tracking player IDs across datasets")

# track which IDs appear in which years
id_tracker = {}

for year in ['2019', '2020', '2021', '2022', '2023']:
    print(f"\nProcessing {year}...")
    df = all_years_data[year]

    players_this_year = 0

    for idx, row in df.iterrows():
        player_id = str(row.get('ID', ''))

        if player_id and player_id != 'nan':
            if player_id not in id_tracker:
                id_tracker[player_id] = {
                    'years': [],
                    'name': row.get('Name'),
                    'nationality': row.get('Nationality'),
                    'position': row.get('Position'),
                    'age_2019': None,
                    'club_2019': None,
                    'club_2020': None,
                    'club_2021': None,
                    'club_2022': None,
                    'club_2023': None,
                    'value_2019': None,
                    'value_2020': None,
                    'value_2021': None,
                    'value_2022': None,
                    'value_2023': None,
                    'data_2023': None
                }

            id_tracker[player_id]['years'].append(year)
            players_this_year += 1

            # store age, club and value from each year
            if year == '2019':
                id_tracker[player_id]['age_2019'] = row.get('Age')
                id_tracker[player_id]['club_2019'] = row.get('Club')
                id_tracker[player_id]['value_2019'] = row.get('Value')
            elif year == '2020':
                id_tracker[player_id]['club_2020'] = row.get('Club')
                id_tracker[player_id]['value_2020'] = row.get('Value')
            elif year == '2021':
                id_tracker[player_id]['club_2021'] = row.get('Club')
                id_tracker[player_id]['value_2021'] = row.get('Value')
            elif year == '2022':
                id_tracker[player_id]['club_2022'] = row.get('Club')
                id_tracker[player_id]['value_2022'] = row.get('Value')
            elif year == '2023':
                id_tracker[player_id]['club_2023'] = row.get('Club')
                id_tracker[player_id]['value_2023'] = row.get('Value')

            # store 2023 data as most recent
            if year == '2023':
                id_tracker[player_id]['data_2023'] = row

    print(f"  tracked {players_this_year:,} players from {year}")

print(f"\nTotal unique player IDs found: {len(id_tracker):,}")

# analyse whether player was there across all years
year_counts = {}
for pid, data in id_tracker.items():
    num_years = len(data['years'])
    year_counts[num_years] = year_counts.get(num_years, 0) + 1

print("\nPlayer retention analysis:")
for years in sorted(year_counts.keys(), reverse=True):
    count = year_counts[years]
    pct = (count / len(id_tracker)) * 100
    print(f"  {years} years: {count:,} players ({pct:.1f}%)")

# filter to only players that are there in all 5 years
print("\nFiltering to players present in all 5 years...")
players_all_years = {
    pid: data for pid, data in id_tracker.items()
    if len(data['years']) == 5 and '2019' in data['years'] and '2020' in data['years']
    and '2021' in data['years'] and '2022' in data['years'] and '2023' in data['years']
}

print(f"Players active in all 5 years: {len(players_all_years):,}")
retention_rate = (len(players_all_years) / len(id_tracker)) * 100
print(f"Retention rate: {retention_rate:.1f}%")

# show random sample of players for verififcation
print("\nRandom sample of players in all five years:")
sample_players = np.random.choice(list(players_all_years.keys()), size=min(15, len(players_all_years)), replace=False)
for i, pid in enumerate(sample_players, 1):
    player = players_all_years[pid]
    print(f"  {i}. {player['name']} ({player['nationality']}) - {player['position']}")

print("\nstage 4 is complete - player matching finished\n")

# Stage 5: Building Master Dataset
print("STAGE 5: Building consolidated master dataset")

print("\nCreating one row per player using data...")
master_records = []

for player_id, player_data in players_all_years.items():
    if player_data['data_2023'] is not None:
        record = player_data['data_2023'].to_dict()

        # add year indicators (all will be 1)
        record['in_2019'] = 1
        record['in_2020'] = 1
        record['in_2021'] = 1
        record['in_2022'] = 1
        record['in_2023'] = 1

        # add age columns
        record['Age in 2019'] = player_data['age_2019']
        record['Age in 2023'] = record.get('Age')

        # add club columns for each year
        record['2019 Club'] = player_data['club_2019']
        record['2020 Club'] = player_data['club_2020']
        record['2021 Club'] = player_data['club_2021']
        record['2022 Club'] = player_data['club_2022']
        record['2023 Club'] = player_data['club_2023']

        # add value columns for each year
        record['2019 Value'] = player_data['value_2019']
        record['2020 Value'] = player_data['value_2020']
        record['2021 Value'] = player_data['value_2021']
        record['2022 Value'] = player_data['value_2022']
        record['2023 Value'] = player_data['value_2023']

        master_records.append(record)

master_df = pd.DataFrame(master_records)

print(f"Master dataset has been created: {len(master_df):,} players")
print(f"Columns in dataset are: {len(master_df.columns)}")

# remove the generic age and club columns and unwanted columns
print("\nRemoving the unnecessary columns...")
removed_cols = []

# add 'Age' and 'Club' to columns to delete since we now have specific year columns
if 'Age' in master_df.columns:
    master_df = master_df.drop('Age', axis=1)
    removed_cols.append('Age')

if 'Club' in master_df.columns:
    master_df = master_df.drop('Club', axis=1)
    removed_cols.append('Club')

for col in columns_to_delete:
    if col in master_df.columns:
        master_df = master_df.drop(col, axis=1)
        removed_cols.append(col)

print(f"Removed {len(removed_cols)} columns:")
for col in removed_cols:
    print(f"  - {col}")

print(f"\nFinal column count: {len(master_df.columns)}")

print("stage 5 finished - master dataset constructed\n")

# STAGE 6: Checking name deduplication and club name formatting and standardisation
print("STAGE 6 - Name deduplication and club standardisation")

print("\nChecking for duplicate names...")

# check for duplicate names
name_counts = master_df['Name'].value_counts()
duplicates = name_counts[name_counts > 1]

if len(duplicates) > 0:
    print(f"Found {len(duplicates)} duplicate names")
    print("\nAdding club to distinguish duplicate names...")

    # for duplicate names, add club to the name
    for dup_name in duplicates.index:
        dup_rows = master_df[master_df['Name'] == dup_name]

        for idx in dup_rows.index:
            # use 2023 club
            club_2023 = master_df.loc[idx, '2023 Club']
            if pd.notna(club_2023):
                master_df.loc[idx, 'Name'] = f"{dup_name} ({club_2023})"

    print(f"  Updated {len(duplicates)} duplicate names with club information")
else:
    print("No duplicate names found")

# standardising the club names
print("\nStandardising club name formatting across all year columns...")

club_columns = ['2019 Club', '2020 Club', '2021 Club', '2022 Club', '2023 Club']

# remove fictional team A.F.C. Richmond - mentioned by Ricardo
fictional_teams = ['A.F.C. Richmond', 'AFC Richmond', 'Richmond']

print("\nRemoving players from fictional teams...")
before_fictional = len(master_df)

# check if any player was at a fictional club in any year to remove
for club_col in club_columns:
    if club_col in master_df.columns:
        for team in fictional_teams:
            master_df = master_df[master_df[club_col] != team]

after_fictional = len(master_df)
removed_fictional = before_fictional - after_fictional

if removed_fictional > 0:
    print(f"  Removed {removed_fictional} players who played for fictional teams")
else:
    print("  No fictional teams found")

def standardise_club_name(name):
    """standardise club names - remove any extra whitespace and make standard"""
    if pd.isna(name):
        return 'Free Agent'

    name = str(name).strip()

    # remove extra whitespace
    name = ' '.join(name.split())

    return name

# Standardising all club columns
for club_col in club_columns:
    if club_col in master_df.columns:
        master_df[club_col] = master_df[club_col].apply(standardise_club_name)

print(f"\nStandardised club names across all {len(club_columns)} year columns")

# Assessing inter club movement statistics for players
print("\nClub movement analysis:")
unique_clubs_per_player = []

for idx, row in master_df.iterrows():
    clubs = [row[col] for col in club_columns if col in master_df.columns and pd.notna(row[col])]
    unique_clubs = len(set(clubs))
    unique_clubs_per_player.append(unique_clubs)

master_df['Total Clubs 2019-2023'] = unique_clubs_per_player

club_changes = pd.Series(unique_clubs_per_player).value_counts().sort_index()

print("\nPlayers by number of different clubs (2019-2023):")
for num_clubs, count in club_changes.items():
    pct = (count / len(master_df)) * 100
    print(f"  {num_clubs} club(s): {count:,} players ({pct:.1f}%)")

print("\nStage 6 complete - names deduplicated and clubs standardised\n")

# Additional step: Need to remove any brackets from names one final time
print("Final name cleaning - removing any remaining brackets...")
def remove_all_brackets(name):
    """Remove any brackets and content inside them from names"""
    if pd.isna(name):
        return name
    name = str(name)

    # Remove everything from ( onwards
    if '(' in name:
        name = name.split('(')[0]

    # Remove everything from [ onwards
    if '[' in name:
        name = name.split('[')[0]

    return name.strip()

master_df['Name'] = master_df['Name'].apply(remove_all_brackets)
print(f"Removed brackets from all {len(master_df)} player names\n")

# Stage 6 part 2 - Converting height and weight to integers
print("Stage 6 part 2 - standardising height and weight measurements")

print("\nconverting heights to centimetres (integers)...")

def convert_height_to_cm(height):
    """convert height to cm whether it's in cm or feet/inches"""
    if pd.isna(height):
        return np.nan

    height = str(height).strip()

    # if it's already in cm format (e.g., "180cm")
    if 'cm' in height:
        try:
            return int(height.replace('cm', '').strip())
        except:
            return np.nan

    # if it's in feet and inches (e.g., "5'11" or "5'11\"")
    if "'" in height or "'" in height:
        try:
            # clean up the string
            height = height.replace('"', '').replace('"', '').strip()

            # split by the apostrophe
            if "'" in height:
                parts = height.split("'")
            else:
                parts = height.split("'")

            feet = int(parts[0].strip())
            inches = int(parts[1].strip()) if len(parts) > 1 and parts[1].strip() else 0

            # convert to cm (1 foot = 30.48cm, 1 inch = 2.54cm)
            total_cm = (feet * 30.48) + (inches * 2.54)
            return int(round(total_cm))
        except:
            return np.nan

    # try to just convert directly if it's already a number
    try:
        return int(float(height))
    except:
        return np.nan

# apply the conversion
if 'Height' in master_df.columns:
    before_conversion = master_df['Height'].head(10).tolist()
    master_df['Height'] = master_df['Height'].apply(convert_height_to_cm)
    after_conversion = master_df['Height'].head(10).tolist()

    print("sample conversions (first 10 players):")
    for i in range(min(10, len(before_conversion))):
        if pd.notna(before_conversion[i]) and pd.notna(after_conversion[i]):
            print(f"  {before_conversion[i]} → {after_conversion[i]}cm")

    missing_heights = master_df['Height'].isna().sum()
    print(f"\nheights converted. {missing_heights} missing values to be filled later")

print("\nconverting weights to kilograms (integers)...")

def convert_weight_to_kg(weight):
    """convert weight to kg whether it's in kg or lbs"""
    if pd.isna(weight):
        return np.nan

    weight = str(weight).strip()

    # if it's in kg format (e.g., "75kg")
    if 'kg' in weight.lower():
        try:
            return int(weight.lower().replace('kg', '').strip())
        except:
            return np.nan

    # if it's in lbs format (e.g., "165lbs")
    if 'lbs' in weight.lower() or 'lb' in weight.lower():
        try:
            weight_lbs = float(weight.lower().replace('lbs', '').replace('lb', '').strip())
            # convert lbs to kg (1 lb = 0.453592 kg)
            weight_kg = weight_lbs * 0.453592
            return int(round(weight_kg))
        except:
            return np.nan

    # try to just convert directly if it's already a number
    try:
        return int(float(weight))
    except:
        return np.nan

# apply the conversion
if 'Weight' in master_df.columns:
    before_conversion = master_df['Weight'].head(10).tolist()
    master_df['Weight'] = master_df['Weight'].apply(convert_weight_to_kg)
    after_conversion = master_df['Weight'].head(10).tolist()

    print("sample conversions (first 10 players):")
    for i in range(min(10, len(before_conversion))):
        if pd.notna(before_conversion[i]) and pd.notna(after_conversion[i]):
            print(f"  {before_conversion[i]} → {after_conversion[i]}kg")

    missing_weights = master_df['Weight'].isna().sum()
    print(f"\nweights converted. {missing_weights} missing values to be filled later")

print("\nstage 6 Part Two complete - heights in cm and weights in kg as integers\n")

# Stage 6 part 3 - standardising all financial columns (wages and values) to full integers
print("Stage 6 Part Three - converting financial values to full integers")

def parse_financial_value(value):
    """convert financial strings to full integer values"""
    if pd.isna(value):
        return np.nan

    value = str(value).strip()

    # remove euro symbol and any whitespace
    value = value.replace('€', '').replace(',', '').strip()

    # handle millions (M, m, mn, Mn)
    if 'M' in value or 'm' in value:
        value = value.replace('M', '').replace('m', '').replace('n', '').strip()
        try:
            return int(float(value) * 1_000_000)
        except:
            return np.nan

    # handle thousands (K, k)
    if 'K' in value or 'k' in value:
        value = value.replace('K', '').replace('k', '').strip()
        try:
            return int(float(value) * 1_000)
        except:
            return np.nan

    # try to convert directly if it's already a number
    try:
        return int(float(value))
    except:
        return np.nan

# list the financial columns to convert
financial_columns = [
    'Wage',
    '2019 Value',
    '2020 Value',
    '2021 Value',
    '2022 Value',
    '2023 Value'
]

print("\nconverting financial columns to full euro values...")

for col in financial_columns:
    if col in master_df.columns:
        # show some examples before conversion
        sample_before = master_df[col].dropna().head(3).tolist()

        # convert the column
        master_df[col] = master_df[col].apply(parse_financial_value)

        # show the converted values
        sample_after = master_df[col].dropna().head(3).tolist()

        print(f"\n{col}:")
        for i in range(min(3, len(sample_before))):
            if i < len(sample_after):
                print(f"  {sample_before[i]} → €{sample_after[i]:,}")

        missing = master_df[col].isna().sum()
        if missing > 0:
            print(f"  ({missing} missing values to be filled later)")

print("\nstage 6 Part Three Complete Now with all financial values converted to full integers\n")

# Stage 7 filling in the missing values
print("STAGE 7: Filling missing values")

print("\nApproach:")
print("  - Categorical fields: logical defaults (e.g., 'Unknown', 'Free Agent')")
print("  - Numerical fields: median values")
print("  - Release Clause: median value")

# categorical columns to assess
categorical_fills = {
    'Nationality': 'Unknown',
    'Position': 'Unknown',
    'Preferred Foot': 'Right',
    'Work Rate': 'Medium/ Medium',
    'Body Type': 'Normal',
}

print("\nFilling categorical columns:")
for col, fill_value in categorical_fills.items():
    if col in master_df.columns:
        before = master_df[col].isna().sum()
        if before > 0:
            master_df[col] = master_df[col].fillna(fill_value)
            print(f"  {col}: filled {before} values with '{fill_value}'")
        else:
            print(f"  {col}: no missing values")

# fill club columns with 'Free Agent' as needed
for club_col in club_columns:
    if club_col in master_df.columns:
        before = master_df[club_col].isna().sum()
        if before > 0:
            master_df[club_col] = master_df[club_col].fillna('Free Agent')
            print(f"  {club_col}: filled {before} values with 'Free Agent'")

# numerical columns - use median for missing
numerical_cols = master_df.select_dtypes(include=['number']).columns

print("\nFilling numerical columns with medians:")
filled_numerical = 0
for col in numerical_cols:
    before = master_df[col].isna().sum()
    if before > 0:
        median_val = master_df[col].median()
        if pd.notna(median_val):
            master_df[col] = master_df[col].fillna(median_val)
            print(f"  {col}: filled {before} values with median ({median_val:.1f})")
            filled_numerical += 1

# specifically handle Release Clause issues
print("\nHandling Release Clause specifically:")
if 'Release Clause' in master_df.columns:
    before_release = master_df['Release Clause'].isna().sum()
    if before_release > 0:
        # parse release clause to numeric if needed
        def parse_release_clause(val):
            if pd.isna(val):
                return np.nan
            val = str(val).replace('€', '').strip()
            if 'M' in val:
                return float(val.replace('M', '')) * 1_000_000
            elif 'K' in val:
                return float(val.replace('K', '')) * 1_000
            try:
                return float(val)
            except:
                return np.nan

        master_df['Release Clause Numeric'] = master_df['Release Clause'].apply(parse_release_clause)
        median_release = master_df['Release Clause Numeric'].median()

        if pd.notna(median_release):
            master_df['Release Clause Numeric'] = master_df['Release Clause Numeric'].fillna(median_release)

            # convert back to formatted string
            def format_release_clause(val):
                if pd.isna(val):
                    return np.nan
                if val >= 1_000_000:
                    return f"€{val/1_000_000:.1f}M"
                elif val >= 1_000:
                    return f"€{val/1_000:.0f}K"
                else:
                    return f"€{val:.0f}"

            master_df['Release Clause'] = master_df['Release Clause Numeric'].apply(format_release_clause)
            master_df = master_df.drop('Release Clause Numeric', axis=1)

            after_release = master_df['Release Clause'].isna().sum()
            print(f"  Release Clause: filled {before_release} values with median (€{median_release/1_000_000:.1f}M)")
        else:
            master_df = master_df.drop('Release Clause Numeric', axis=1)
            print(f"  Release Clause: no valid median found")
    else:
        print(f"  Release Clause: no missing values")

if filled_numerical == 0:
    print("  No other numerical columns required filling")

print(f"\nstage 7 complete - missing values addressed\n")

# Stage 8 is final data validation
print("Stage 8 - Final data quality assessment")

missing_after = master_df.isna().sum()
missing_pct_after = (missing_after / len(master_df) * 100).round(2)

missing_report_after = pd.DataFrame({
    'column': missing_after.index,
    'missing_count': missing_after.values,
    'missing_percentage': missing_pct_after.values
}).sort_values('missing_percentage', ascending=False)

print("\nTop 15 columns with missing data (after cleaning):")
print(missing_report_after.head(15).to_string(index=False))

total_cells_after = len(master_df) * len(master_df.columns)
total_missing_after = master_df.isna().sum().sum()
overall_pct_after = (total_missing_after / total_cells_after) * 100

print(f"\nFinal data completeness:")
print(f"  Total cells: {total_cells_after:,}")
print(f"  Missing cells: {total_missing_after:,}")
print(f"  Missing percentage: {overall_pct_after:.2f}%")
print(f"  Complete data: {100 - overall_pct_after:.2f}%")

# comparison
improvement = overall_pct_before - overall_pct_after
print("\nData quality improvement summary:")
print(f"  Before cleaning:  {overall_pct_before:.2f}% missing")
print(f"  After Cleaning:   {overall_pct_after:.2f}% missing")
print(f"  Improvement:      {improvement:.2f}% reduction")
print(f"  Data completeness increased by: {improvement:.2f} percentage points")

print("\nstage 8 is complete - validation finished\n")

# STAGE 9: Data Reduction Summary and Export of cleaned data
print("STAGE 9 - Data reduction summary and export")

print("\nDataset consolidation statistics:")
print(f"  Original total records (across all 5 years): {total_players_loaded:,}")
print(f"  Final consolidated records (players in all 5 years): {len(master_df):,}")

reduction = total_players_loaded - len(master_df)
reduction_pct = (reduction / total_players_loaded) * 100

print(f"\nData reduction:")
print(f"  Records reduced by: {reduction:,}")
print(f"  Reduction percentage: {reduction_pct:.1f}%")
print(f"  Final dataset is {100 - reduction_pct:.1f}% of original size")

print("\nThis shows the dataset is now much more manageable:")
print(f"  Before: {total_players_loaded:,} rows (duplicates across years)")
print(f"  After: {len(master_df):,} rows (unique players in all years)")
print(f"  Reduction: {reduction_pct:.1f}% fewer records")

output_file = '/content/drive/MyDrive/FIFA CLEAN DATA GROUP B.csv'
master_df.to_csv(output_file, index=False, encoding='utf-8')

print(f"\nMaster dataset saved: {output_file}")
print(f"\nFinal dataset specifications:")
print(f"  Total players: {len(master_df):,}")
print(f"  Total columns: {len(master_df.columns)}")
print(f"  Years covered: 2019-2023 (5 years)")
print(f"  Selection criteria: players active in ALL 5 years")
print(f"  Names formatted: 'F. Surname' format (e.g., 'G. Bale')")
print(f"  Age columns: 'Age in 2019' and 'Age in 2023' for clarity")
print(f"  Club columns: 2019-2023 club history tracked")
print(f"  Total Clubs: column showing number of different clubs per player")
print(f"  Position format: readable names (e.g., 'Left Back', 'Striker')")
print(f"  Data completeness: {100 - overall_pct_after:.2f}%")
print(f"  Release Clause: missing values filled with median")

print("\n" + "="*60)
print("ALL STAGES COMPLETE")
print("="*60)
print("\nDataset ready for analysis by group B")

installing fuzzywuzzy...
installation complete

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

FIFA Master Dataset Creation
Players Active 2019-2023

Stage 1: Configuration and setup
Data sources configured: 5 years (2019-2023)
Columns to be removed: 18
player position mapping configured: 27 positions
Initial work complete - all data ready

STAGE 2 - Sourcing all raw data from our downloaded files on repo on git

Loading FIFA 2019...
  raw data: 17,943 rows, 64 columns
  cleaned 20 text columns (removed HTML tags)
  formatted player names to 'F. Surname' format
  converted position codes to readable names
  FIFA 2019 loaded successfully

Loading FIFA 2020...
  raw data: 17,104 rows, 65 columns
  cleaned 20 text columns (removed HTML tags)
  formatted player names to 'F. Surname' format
  converted position codes to readable names
  FIFA 2020 loaded successfully

Loading FIFA 2021...
  raw data: 17,108 ro